In [21]:
import pandas as pd
import numpy as np
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
from tslearn.clustering import TimeSeriesKMeans
import matplotlib.pyplot as plt

# Read 2020 data

In [2]:
tmp20 = pd.read_csv("data/CE1929.txt", sep=";", header=None)
tmp20.columns=["Company", "TEÁOR", "Meta", "Value"]
df20 = pd.pivot_table(tmp20, values="Value", index=["Company"], columns="Meta")
df20.columns = "20"+df20.columns

# Read 2019 data

In [3]:
tmp19 = pd.read_csv("data/CE1829.txt", sep=";", header=None)
tmp19.columns=["Company", "TEÁOR", "Meta", "Value"]
df19 = pd.pivot_table(tmp19, values="Value", index=["Company"], columns="Meta")
df19.columns = "19"+df19.columns

# Read 2018 data

In [4]:
tmp18 = pd.read_csv("data/CE1729.txt", sep=";", header=None)
tmp18.columns=["Company", "TEÁOR", "Meta", "Value"]
df18 = pd.pivot_table(tmp18, values="Value", index=["Company"], columns="Meta")
df18.columns = "18"+df18.columns

# Read 2017 data

In [6]:
tmp17 = pd.read_csv("data/CE1629.txt", sep=";", header=None)
tmp17.columns=["Company", "TEÁOR", "Meta", "Value"]
stringMeta = ['TAB091', 'TAB088', 'TAB226', 'TAB229']
stringIndex = tmp17[tmp17['Meta'].isin(stringMeta)].index
tmp17.drop(stringIndex, inplace = True)
df17 = pd.pivot_table(tmp17, values="Value", index=["Company"], columns="Meta")
df17.columns = "17"+df17.columns

# Read 2016 data

In [7]:
tmp16 = pd.read_csv("data/CE1529.txt", sep=";", header=None)
tmp16.columns=["Company", "TEÁOR", "Meta", "Value"]
stringMeta = ['TAB091', 'TAB088', 'TAB226', 'TAB229']
stringIndex = tmp16[tmp16['Meta'].isin(stringMeta)].index
tmp16.drop(stringIndex, inplace = True)
df16 = pd.pivot_table(tmp16, values="Value", index=["Company"], columns="Meta")
df16.columns = "16"+df16.columns

# Read 2015 data

In [8]:
tmp15 = pd.read_csv("data/CE1429.txt", sep=";", header=None)
tmp15.columns=["Company", "TEÁOR", "Meta", "Value"]
tmp15["Value"] = pd.to_numeric(tmp15["Value"], errors='coerce')
tmp15.dropna(inplace = True)
df15 = pd.pivot_table(tmp15, values="Value", index=["Company"], columns="Meta")
df15.columns = "15"+df15.columns

# Read 2014 data

In [9]:
tmp14 = pd.read_csv("data/CE1329.txt", sep=";", header=None)
tmp14.columns=["Company", "TEÁOR", "Meta", "Value"]
tmp14["Value"] = pd.to_numeric(tmp14["Value"], errors='coerce')
tmp14.dropna(inplace = True)
df14 = pd.pivot_table(tmp14, values="Value", index=["Company"], columns="Meta")
df14.columns = "14"+df14.columns

# Merge all years

In [10]:
years = [df14, df15, df16, df17, df18, df19, df20]
df = pd.concat(years, axis = 1, join ="inner")
df["TEÁOR"] = tmp16["TEÁOR"]
df["sector"] = df["TEÁOR"].astype(str).str[0:2]

# Filter small companies

In [11]:
yearsString = ["14", "15", "16", "18", "19", "20"]
for y in yearsString:
    smallCompaniesIndex = df[df[y+"TAH082"] < 10].index
    df.drop(smallCompaniesIndex, inplace = True)
smallCompaniesIndex = df[df["17TAH183"] < 10].index
df.drop(smallCompaniesIndex, inplace = True)
df.fillna(0, inplace = True)

# Drop outliers

In [ ]:
df.drop(df[
        (df["14Productivity"]>df["14Productivity"].quantile(.97)) |
        (df["14Productivity"]<df["14Productivity"].quantile(.03)) |
        (df["15Productivity"]>df["15Productivity"].quantile(.97)) |
        (df["15Productivity"]<df["15Productivity"].quantile(.03)) |
        (df["16Productivity"]>df["16Productivity"].quantile(.97)) |
        (df["16Productivity"]<df["16Productivity"].quantile(.03)) |
        (df["17Productivity"]>df["17Productivity"].quantile(.97)) |
        (df["17Productivity"]<df["17Productivity"].quantile(.03)) |
        (df["18Productivity"]>df["18Productivity"].quantile(.97)) |
        (df["18Productivity"]<df["18Productivity"].quantile(.03)) |
        (df["19Productivity"]>df["19Productivity"].quantile(.97)) |
        (df["19Productivity"]<df["19Productivity"].quantile(.03)) |    
        (df["20Productivity"]>df["20Productivity"].quantile(.97)) |
        (df["20Productivity"]<df["20Productivity"].quantile(.03))
].index, inplace=True)
df.drop(df[(df["14TAH033"]<0) | (df["20TAH033"]<0) | (df["15TAH033"]<0) | (df["16TAH033"]<0) | (df["17TAH033"]<0) | (df["18TAH033"]<0) | (df["19TAH033"]<0)].index, inplace=True)

# Calculate productivity

In [12]:
for y in yearsString:
    df[y+"ProductivityNet"] = (df[y+"TAC002"]-df[y+"TAC007"]+df[y+"TAC006"]-df[y+"TAC016"]) / df[y+"TAH082"]
    df[y+"Productivity"] = (df[y+"TAC002"]-df[y+"TAC007"]+df[y+"TAC006"]) / df[y+"TAH082"]
    zeroProd = df[df[y+"Productivity"] == 0]
    df.drop(df[df[y+"Productivity"] == 0].index, inplace =True)
    df.drop(df[(df[y+"Productivity"] == np.inf) | (df[y+"Productivity"] == -np.inf)].index, inplace =True)
    df.drop(df[pd.isna(df[y+"Productivity"])].index, inplace =True)
    df.drop(df[(df[y+"TAH033"]<0)].index, inplace=True)
    df.drop(columns = [y+"TAC002", y+"TAC007", y+"TAC006", y+"TAC016", y+"TAH082"], inplace=True)
    
df["17ProductivityNet"] = (df["17TAC002"]-df["17TAC007"]+df["17TAC006"]-df["17TAC016"]) / df["17TAH183"]
df["17Productivity"] = (df["17TAC002"]-df["17TAC007"]+df["17TAC006"]) / df["17TAH183"]
zeroProd = df[df["17Productivity"] == 0]
df.drop(df[df["17Productivity"] == 0].index, inplace =True)
df.drop(df[(df["17Productivity"] == np.inf) | (df["17Productivity"] == -np.inf)].index, inplace =True)
df.drop(df[pd.isna(df["17Productivity"])].index, inplace =True)
df.drop(df[(df["17TAH033"]<0)].index, inplace=True)
df.drop(columns = ["17TAC002", "17TAC007", "17TAC006", "17TAC016", "17TAH082"], inplace=True)

for i in range(14, 20): 
    df[str(i)+"c"+str(i+1)+"Productivity"] = (df[str(i+1)+"Productivity"] - df[str(i)+"Productivity"])/abs(df[str(i)+"Productivity"])

In [14]:
penzugyi = [64, 65, 66]
df.drop(df[df["sector"].isin(penzugyi)].index, inplace=True)

In [53]:
df.describe()

Meta,14ARBKOD,14MKE106,14MKU106,14TAA004,14TAB001,14TAB004,14TAB005,14TAB007,14TAB008,14TAB012,...,20ProductivityNet,20Productivity,17ProductivityNet,17Productivity,14c15Productivity,15c16Productivity,16c17Productivity,17c18Productivity,18c19Productivity,19c20Productivity
count,13869.000000,13869.000000,13869.000000,1.386900e+04,1.386900e+04,1.386900e+04,1.386900e+04,13869.000000,1.386900e+04,1.386900e+04,...,1.386900e+04,1.386900e+04,1.386900e+04,1.386900e+04,13869.000000,13869.000000,13869.000000,13869.000000,13869.000000,13869.000000
mean,0.003749,1.683539,1.692191,-3.111633e+03,1.212728e+05,2.224362e+05,1.747729e+05,136.523614,7.941303e+04,1.664800e+04,...,8.670828e+03,1.012612e+04,6.680606e+03,7.951720e+03,0.401254,0.267833,0.423965,0.554072,0.247986,2.260266
std,0.091381,0.999208,1.002627,7.691482e+04,2.847679e+06,2.806407e+06,1.663874e+06,14358.777089,1.195562e+06,1.813116e+05,...,3.245648e+04,3.779515e+04,2.400650e+04,3.012228e+04,5.231296,4.594533,24.120663,34.924154,7.268882,228.927023
min,0.000000,0.000000,0.000000,-2.433726e+06,-5.200612e+07,0.000000e+00,0.000000e+00,-975965.000000,-5.207106e+07,0.000000e+00,...,-1.664293e+06,-1.664293e+06,-1.015944e+06,-1.015860e+06,-37.646035,-69.694100,-357.954035,-1273.526316,-522.951720,-150.100000
25%,0.000000,1.000000,1.000000,-8.260000e+02,3.541000e+03,6.313000e+03,5.542000e+03,0.000000,3.643000e+03,3.640000e+02,...,3.318231e+03,4.018323e+03,2.440700e+03,2.972909e+03,-0.064676,-0.093809,-0.107251,-0.050058,-0.051799,-0.075660
50%,0.000000,2.000000,2.000000,0.000000e+00,1.969400e+04,2.283400e+04,1.769600e+04,0.000000,1.517600e+04,1.518000e+03,...,5.563816e+03,6.628236e+03,4.178368e+03,5.030600e+03,0.082293,0.050540,0.040542,0.089437,0.086002,0.066572
75%,0.000000,2.000000,2.000000,0.000000e+00,7.080000e+04,7.286800e+04,5.593600e+04,0.000000,5.493200e+04,5.493000e+03,...,9.284000e+03,1.079539e+04,7.221208e+03,8.518400e+03,0.289509,0.226212,0.198321,0.271304,0.257101,0.224410
max,3.000000,3.000000,3.000000,6.906000e+06,2.816785e+08,2.289199e+08,9.401780e+07,501118.000000,5.289952e+07,1.000591e+07,...,1.870100e+06,2.328904e+06,1.444264e+06,2.111312e+06,368.702303,342.791045,2582.916667,3831.195122,566.781040,26931.800000


In [13]:
df.to_pickle("data/dataframeAlap")